In [1]:
import os
import glob
import numpy as np
import pandas as pd
import mc3
import MC3_BASELINES as baselines

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import rc, cm

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
rc('font',**{'size': 12}) # 'family':'Avenir'
plt.rcParams['pdf.fonttype'] = 42

In [2]:
# Get working paths 
path_parent = os.path.dirname(os.getcwd())
path_beg = os.getcwd() + '/'
path_input = os.getcwd() + '/Inputs/'
output_dir = ["FIGURES", "PLOTFILES", "NPZFILES", "LOGFILES", "FINALDATA"] 

# Create output directories if not in existence 
for ii in range(len(output_dir)):
    if not os.path.exists(path_beg + output_dir[ii]):
       os.makedirs(path_beg + output_dir[ii], exist_ok=True)

# Change paths to direct to folder with SampleSpectra -- last bit should be whatever your folder with spectra is called. 
PATHS = [path_input+'SampleSpectra/Fuego/', path_input+'SampleSpectra/Standards/', path_input+'SampleSpectra/Fuego1974RH/', path_input+'SampleSpectra/SIMS/', path_input+'SampleSpectra/SIMSStd_2/']

# Put ChemThick file in Inputs. Direct to what your ChemThick file is called. 
CHEMTHICK_PATH = [path_input+'FuegoChemThick.csv', path_input+'StandardChemThick.csv', path_input+'DanRHChemThick.csv', path_input+'SIMSChemThick.csv', path_input+'SIMSChemThick_2.csv']

# Change last value in list to be what you want your output directory to be called. 
INPUT_PATHS = [[path_input+'Baseline_AvgPCA.csv', path_input+"Water_Peak_1635_All.csv", path_beg, 'FUEGO_F'],
                [path_input+'Baseline_AvgPCA.csv', path_input+"Water_Peak_1635_All.csv", path_beg, 'STD_F'], 
                [path_input+'Baseline_AvgPCA.csv', path_input+"Water_Peak_1635_All.csv", path_beg, 'FRH_F'],
                [path_input+'Baseline_AvgPCA.csv', path_input+"Water_Peak_1635_All.csv", path_beg, 'SIMS_F'],
                [path_input+'Baseline_AvgPCA.csv', path_input+"Water_Peak_1635_All.csv", path_beg, 'SIMS2_F']]

# Change to be what you want the prefix of your output files to be. 
OUTPUT_PATH = ['F18', 'STD', 'FRH', 'SIMSSTD', 'SIMSSTD2']

In [3]:
REF_PATH = path_input + '/ReflectanceSpectra/FuegoOl/'
REF_FILES = glob.glob(REF_PATH + "*")
REF_FILES.sort()

REF_DFS_FILES, REF_DFS_DICT = baselines.Load_SampleCSV(REF_FILES, wn_high = 2800, wn_low = 2000)

# Use DHZ parameterization of olivine reflectance index. 
n_ol = baselines.ReflectanceIndex(0.72)

REF_FUEGO = baselines.ThicknessProcessing(REF_DFS_DICT, n = n_ol, wn_high = 2700, wn_low = 2100, remove_baseline = True, plotting = False, phaseol = True)

In [4]:
REF_PATH = path_input + '/ReflectanceSpectra/rf_ND70/'
REF_FILES = glob.glob(REF_PATH + "*")
REF_FILES.sort()

REF_DFS_FILES, REF_DFS_DICT = baselines.Load_SampleCSV(REF_FILES, wn_high = 2850, wn_low = 1700)

# n=1.546 in the range of 2000-2700 cm^-1 following Nichols and Wysoczanski, 2007 for basaltic glass
n_gl = 1.546

REF_FUEGO = baselines.ThicknessProcessing(REF_DFS_DICT, n = n_gl, wn_high = 2850, wn_low = 1700, remove_baseline = True, plotting = False, phaseol = False)

In [5]:
simsno = 3 

PATH = PATHS[simsno]
FILES = glob.glob(PATH + "*")
FILES.sort()

MICOMP, THICKNESS = baselines.Load_ChemistryThickness(CHEMTHICK_PATH[simsno])

DFS_FILES, DFS_DICT = baselines.Load_SampleCSV(FILES, wn_high = 5500, wn_low = 1000)
DF_OUTPUT, FAILURES = baselines.Run_All_Spectra(DFS_DICT, INPUT_PATHS[simsno])


DF_OUTPUT.to_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[simsno] + '_DF.csv')

# DF_OUTPUT = pd.read_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[simsno] + '_DF.csv', index_col = 0)

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = baselines.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[simsno] + '_H2OCO2.csv')
DENSITY_EPSILON.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[simsno] + '_DensityEpsilon.csv')
MEGA_SPREADSHEET


::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  Multi-core Markov-chain Monte Carlo (mc3).
  Version 3.0.13.
  Copyright (c) 2015-2023 Patricio Cubillos and collaborators.
  mc3 is open-source software under the MIT license (see LICENSE).
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

Least-squares best-fitting parameters:
  [ 2.02473245e+00  1.87020955e+00  2.86142095e-01 -2.54504598e-02
 -3.61383535e-01  1.43166755e+03  3.52028432e+01  3.49692362e-01
  1.52368725e+03  3.11145163e+01  3.71188423e-01  3.53164253e-01
 -2.03646625e-01  4.14774744e-01  3.03192461e-05  1.12919806e+00]

Yippee Ki Yay Monte Carlo!
Start MCMC chains  (Fri Mar  3 17:36:16 2023)

[:         ]  10.0% completed  (Fri Mar  3 17:36:23 2023)
Out-of-bound Trials:
[  1 249  19 564 249  73  90   0  91  52   0   1   2 227   0   0]
Best Parameters: (chisq=77.7379)
[ 2.02473245e+00  1.87020955e+00  2.86142095e-01 -2.54504598e-02
 -3.61383535e-01  1.43166755e+03  3.5202843

,H2OT_MEAN,H2OT_STD,H2OT_3550_M,H2OT_3550_STD,H2OT_3550_SAT,H2Om_1635_BP,H2Om_1635_STD,CO2_MEAN,CO2_STD,CO2_1515_BP,...,CO2_1430_BP,CO2_1430_STD,H2Om_5200_M,H2Om_5200_STD,OH_4500_M,OH_4500_STD,PH_5200_S2N,PH_4500_S2N,ERR_5200,ERR_4500
ND70_02_01_06032022_150x150_sp1,1.879066,0.190412,2.821279,0.217839,*,0.664208,0.121356,1328.070603,42.742521,1353.240783,...,1274.872641,59.259285,0.719281,0.21097,1.19503,0.35687,9.035935,9.666112,-,-
ND70_02_01_06032022_150x150_sp2,1.944956,0.197471,2.82231,0.21785,*,0.68487,0.125221,1341.091665,43.047314,1366.770053,...,1289.076959,59.75355,0.545321,0.157161,1.240988,0.370053,8.640499,14.06714,-,-
ND70_02_01_06032022_150x150_sp3,1.783725,0.177219,2.818011,0.217557,*,0.660478,0.120652,1364.048723,43.699709,1389.331414,...,1307.13407,60.479315,0.70833,0.207222,1.102565,0.329018,6.542633,12.275726,-,-
ND70_03_01_06032022_150x150_sp1,2.596221,0.250911,2.530655,0.197477,*,1.116795,0.208277,1799.448095,57.005548,1863.107598,...,1738.457326,78.616483,1.013587,0.30446,1.481352,0.456974,9.218763,11.912369,-,-
ND70_03_01_06032022_150x150_sp2,2.781793,0.268320,5.60941,0.437337,*,1.163778,0.216874,2021.45994,63.854936,2024.505565,...,1893.657718,85.446305,1.013036,0.303332,1.532174,0.472829,11.620757,12.624295,-,-
ND70_03_01_06032022_150x150_sp3,2.658923,0.254494,5.644396,0.440282,*,1.130408,0.210612,2004.067515,63.346119,1997.885561,...,1879.549433,84.918008,0.980969,0.294523,1.441811,0.4454,9.356311,13.720353,-,-
ND70_04_02_06032022_150x150_sp1,2.771658,0.244432,5.491872,0.42541,*,1.264966,0.228327,3077.092551,95.992585,3090.144182,...,2881.813342,127.461446,0.914975,0.257288,1.424622,0.416548,7.457396,11.450836,-,-
ND70_04_02_06032022_150x150_sp2,3.027419,0.264581,5.496119,0.42562,*,1.413128,0.25447,3376.824397,105.091845,3401.700407,...,3170.944128,139.894043,1.290134,0.362352,1.533154,0.447848,8.750288,13.842134,-,-
ND70_04_02_06032022_150x150_sp3,3.000831,0.259044,3.145139,0.243973,*,1.479921,0.266628,3352.98225,104.316996,3473.982373,...,3221.195003,141.849916,1.2647,0.355301,1.516082,0.443117,10.810417,13.442954,-,-
ND70_05_02_06032022_150x150_sp1,4.168685,0.357146,4.267484,0.323653,*,2.481155,0.469362,8090.721851,229.781417,8607.722823,...,7555.720014,303.11231,2.053071,0.640118,1.682892,0.538031,10.696872,12.864924,-,-


In [6]:
simsno2 = 4

PATH = PATHS[simsno2]
FILES = glob.glob(PATH + "*")
FILES.sort()

MICOMP, THICKNESS = baselines.Load_ChemistryThickness(CHEMTHICK_PATH[simsno2])

DFS_FILES, DFS_DICT = baselines.Load_SampleCSV(FILES, wn_high = 5500, wn_low = 1000)
DF_OUTPUT, FAILURES = baselines.Run_All_Spectra(DFS_DICT, INPUT_PATHS[simsno2])
DF_OUTPUT.to_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[simsno2] + '_DF.csv')

# DF_OUTPUT = pd.read_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[simsno2] + '_DF.csv', index_col = 0)

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = baselines.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[simsno2] + '_H2OCO2.csv')
DENSITY_EPSILON.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[simsno2] + '_DensityEpsilon.csv')
MEGA_SPREADSHEET


::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  Multi-core Markov-chain Monte Carlo (mc3).
  Version 3.0.13.
  Copyright (c) 2015-2023 Patricio Cubillos and collaborators.
  mc3 is open-source software under the MIT license (see LICENSE).
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

Least-squares best-fitting parameters:
  [ 6.41441089e-01  1.16020852e+00  1.45840638e-01  2.49613052e-01
 -1.61006029e-01  1.43146891e+03  3.69854098e+01  1.10837392e-01
  1.52467183e+03  3.15926962e+01  1.14182783e-01  1.02695681e-01
 -5.48357532e-02  1.23509348e-01  2.57413859e-04  1.70104625e-01]

Yippee Ki Yay Monte Carlo!
Start MCMC chains  (Fri Mar  3 18:08:29 2023)

[:         ]  10.0% completed  (Fri Mar  3 18:08:41 2023)
Out-of-bound Trials:
[  16    9   15   20   11  173 5281    2  313  302    1    3    3    0
    0    0]
Best Parameters: (chisq=18.4540)
[ 6.41441089e-01  1.16020852e+00  1.45840638e-01  2.49613052e-01
 -1.61006029e-01  1.431468

,H2OT_MEAN,H2OT_STD,H2OT_3550_M,H2OT_3550_STD,H2OT_3550_SAT,H2Om_1635_BP,H2Om_1635_STD,CO2_MEAN,CO2_STD,CO2_1515_BP,...,CO2_1430_BP,CO2_1430_STD,H2Om_5200_M,H2Om_5200_STD,OH_4500_M,OH_4500_STD,PH_5200_S2N,PH_4500_S2N,ERR_5200,ERR_4500
ND70_02-01_30June2022_150x150_sp1,3.853961,0.607907,3.853961,0.607907,-,1.011393,0.239432,2147.891906,229.642542,2179.824764,...,2115.959048,325.02597,1.134248,0.384434,1.09816,0.372286,7.753115,11.927947,-,-
ND70_02-01_30June2022_150x150_sp2,3.727927,0.589347,3.727927,0.589347,-,0.986785,0.233995,1838.345294,202.035211,1898.080212,...,1778.610376,282.625262,0.299566,0.099221,2.93305,1.040349,3.872114,13.553811,*,-
ND70_02-01_30June2022_150x150_sp3,3.427590,0.482610,3.42759,0.48261,-,0.879875,0.197507,1742.278091,166.930276,1799.868436,...,1684.687747,234.611893,2.187381,0.745077,0.729587,0.239851,19.547807,5.259082,-,-
ND70_02-01_30June2022_150x150_sp4,4.005539,0.632303,4.005539,0.632303,-,1.075633,0.25408,1963.002461,211.778566,2042.541017,...,1883.463905,292.895696,0.893176,0.310647,4.9633,1.726205,2.633839,3.518766,*,*
ND70_03-01_30June2022_150x150_sp1,3.329736,0.427354,3.329736,0.427354,-,1.166663,0.25322,1901.583145,158.988222,1934.192518,...,1868.973772,225.37676,1.231806,0.401504,2.071944,0.70159,17.132012,9.279202,-,-
ND70_03-01_30June2022_150x150_sp2,3.785508,0.533914,3.785508,0.533914,-,1.325335,0.298624,2190.241189,202.560028,2233.297533,...,2147.184844,285.162104,0.934881,0.311793,2.566596,0.898997,6.871001,9.459134,-,-
ND70_03-01_30June2022_150x150_sp3,4.138584,0.583339,4.138584,0.583339,-,1.461392,0.329534,2365.073434,217.679847,2418.321976,...,2311.824892,305.415565,0.75757,0.250297,3.183175,1.110095,3.13071,11.721556,*,-
ND70_03-01_30June2022_150x150_sp4,5.023326,0.792541,5.023326,0.792541,-,1.622611,0.377938,2823.63674,295.346243,2940.310167,...,2706.963312,403.488404,2.109863,0.684843,4.640547,1.54746,1.930331,3.579585,*,*
ND70_0503_29June2022_100x60_256s_sp1,5.223795,0.737682,5.223795,0.737682,-,1.845726,0.422082,1418.458749,141.00447,1446.817785,...,1390.099713,201.475015,3.196575,1.123787,2.006656,0.700469,7.591094,7.324789,-,-
ND70_0503_29June2022_95x80_256s_sp2,4.503154,0.579542,4.503154,0.579542,-,1.604693,0.352915,1001.383025,93.229968,1075.565325,...,927.200725,127.446272,1.792367,0.601257,1.755768,0.600794,8.795233,6.883495,-,-


In [7]:
stdno = 1

PATH = PATHS[stdno]
FILES = glob.glob(PATH + "*")
FILES.sort()

MICOMP, THICKNESS = baselines.Load_ChemistryThickness(CHEMTHICK_PATH[stdno])

DFS_FILES, DFS_DICT = baselines.Load_SampleCSV(FILES, wn_high = 5500, wn_low = 1000)
DF_OUTPUT, FAILURES = baselines.Run_All_Spectra(DFS_DICT, INPUT_PATHS[stdno])
DF_OUTPUT.to_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[stdno] + '_DF.csv')

# DF_OUTPUT = pd.read_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[stdno] + '_DF.csv', index_col = 0)

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = baselines.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[stdno] + '_H2OCO2.csv')
DENSITY_EPSILON.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[stdno] + '_DensityEpsilon.csv')

def STD_DF_MOD(MEGA_SPREADSHEET):
    STD_VAL = pd.DataFrame(index = MEGA_SPREADSHEET.index, columns = ['H2O_EXP', 'H2O_EXP_STD', 'CO2_EXP', 'CO2_EXP_STD'])

    for j in MEGA_SPREADSHEET.index: 
            
        if '21ALV1846' in j: 
            H2O_EXP= 1.89
            H2O_EXP_STD = 0.19
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif '23WOK5-4' in j: 
            H2O_EXP = 1.6
            H2O_EXP_STD = 0.16
            CO2_EXP = 64	
            CO2_EXP_STD = 6.4
        elif 'ALV1833-11' in j: 
            H2O_EXP = 1.2
            H2O_EXP_STD = 0.12
            CO2_EXP = 102
            CO2_EXP_STD = 10.2
        elif 'CD33_12-2-2' in j: 
            H2O_EXP = 0.27
            H2O_EXP_STD = 0.03
            CO2_EXP = 170
            CO2_EXP_STD = 17
        elif 'CD33_22-1-1' in j: 
            H2O_EXP = 0.49
            H2O_EXP_STD = 0.05
            CO2_EXP = 109
            CO2_EXP_STD = 10.9
        elif 'ETFSR_Ol8' in j: 
            H2O_EXP = 4.16
            H2O_EXP_STD = 0.42
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'Fiege63' in j: 
            H2O_EXP = 3.10
            H2O_EXP_STD = 0.31
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'Fiege73' in j: 
            H2O_EXP = 4.47
            H2O_EXP_STD = 0.45
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'STD_C1' in j: 
            H2O_EXP = 3.26
            H2O_EXP_STD = 0.33
            CO2_EXP = 169
            CO2_EXP_STD = 16.9
        elif 'STD_CN92C_OL2' in j: 
            H2O_EXP = 4.55
            H2O_EXP_STD = 0.46
            CO2_EXP = 270
            CO2_EXP_STD = 27
        elif 'STD_D1010' in j: 
            H2O_EXP = 1.13
            H2O_EXP_STD = 0.11
            CO2_EXP = 139
            CO2_EXP_STD = 13.9
        elif 'STD_ETFS' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'VF74_127-7' in j: 
            H2O_EXP = 3.98
            H2O_EXP_STD = 0.39
            CO2_EXP = 439
            CO2_EXP_STD = 43.9
        elif 'VF74_131-1' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'VF74_131-9' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'VF74_132-1' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'VF74_132-2' in j: 
            H2O_EXP = 3.91
            H2O_EXP_STD = 0.39
            CO2_EXP = 198
            CO2_EXP_STD = 19.8
        elif 'VF74_134D-15' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan
        elif 'VF74_136-3' in j: 
            H2O_EXP = np.nan
            H2O_EXP_STD = np.nan
            CO2_EXP = np.nan
            CO2_EXP_STD = np.nan

        STD_VAL.loc[j] = pd.Series({'H2O_EXP':H2O_EXP,'H2O_EXP_STD':H2O_EXP_STD,'CO2_EXP':CO2_EXP,'CO2_EXP_STD':CO2_EXP_STD})

    MEGA_SPREADSHEET_STD = pd.concat([MEGA_SPREADSHEET, STD_VAL], axis = 1)

    return MEGA_SPREADSHEET_STD

MEGA_SPREADSHEET_STD = STD_DF_MOD(MEGA_SPREADSHEET)

MEGA_SPREADSHEET_STD.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[stdno] + '_H2OCO2_FwSTD.csv')

MEGA_SPREADSHEET_STD


::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  Multi-core Markov-chain Monte Carlo (mc3).
  Version 3.0.13.
  Copyright (c) 2015-2023 Patricio Cubillos and collaborators.
  mc3 is open-source software under the MIT license (see LICENSE).
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

Least-squares best-fitting parameters:
  [ 1.35552143e+00  3.52047737e+00  1.00249803e-01  4.67072495e-01
  1.03168631e-01  1.42166756e+03  3.99999552e+01  1.23059552e-18
  1.52922607e+03  2.56001143e+01  8.36005679e-18  1.75529655e-01
 -1.84764506e-02  1.04670588e-02  1.66024932e-04  7.22002617e-01]

Yippee Ki Yay Monte Carlo!
Start MCMC chains  (Fri Mar  3 18:36:16 2023)

[:         ]  10.0% completed  (Fri Mar  3 18:36:24 2023)
Out-of-bound Trials:
[    0     0     1     5     0 15731  8323 11959 14351  8750  9561     0
     0     0     0     0]
Best Parameters: (chisq=48.5469)
[ 1.35552143e+00  3.52047737e+00  1.00249803e-01  4.67072495e-01
  1.031686

,H2OT_MEAN,H2OT_STD,H2OT_3550_M,H2OT_3550_STD,H2OT_3550_SAT,H2Om_1635_BP,H2Om_1635_STD,CO2_MEAN,CO2_STD,CO2_1515_BP,...,OH_4500_M,OH_4500_STD,PH_5200_S2N,PH_4500_S2N,ERR_5200,ERR_4500,H2O_EXP,H2O_EXP_STD,CO2_EXP,CO2_EXP_STD
21ALV1846-9_44x44_s1_EP_103um,1.387128,0.108395,1.387128,0.108395,-,0.347536,0.058335,0.0,3.942771,0.0,...,0.83727,0.214241,10.02839,7.156581,-,-,1.89,0.19,NaN,NaN
21ALV1846-9_44x44_s2_EP_103um,1.490365,0.116303,1.490365,0.116303,-,0.354715,0.059481,0.0,4.052034,0.0,...,0.803276,0.205283,7.212676,10.331606,-,-,1.89,0.19,NaN,NaN
21ALV1846-9_44x44_s3_EP_103um,1.398400,0.109117,1.3984,0.109117,-,0.347982,0.058277,0.0,3.836463,0.0,...,0.832492,0.213172,10.619909,8.536247,-,-,1.89,0.19,NaN,NaN
23WOK5-4_44x44_s1_EP_147um,1.043671,0.078351,1.043671,0.078351,-,0.211256,0.03552,30.917425,8.541187,31.805809,...,0.540746,0.139538,6.536997,9.246675,-,-,1.6,0.16,64.0,6.4
23WOK5-4_44x44_s2_EP_147um,0.974799,0.073339,0.974799,0.073339,-,0.208961,0.035193,40.045603,9.007794,51.730308,...,0.55573,0.143383,7.202544,10.437003,-,-,1.6,0.16,64.0,6.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VF74_134D-15_101220_256s_15x25_b,6.318766,0.932838,6.318766,0.932838,-,4.423068,0.860117,1241.126131,140.099225,1156.300521,...,3.194434,0.799969,4.872235,4.565704,-,-,NaN,NaN,NaN,NaN
VF74_134D-15_101220_256s_15x25_c,6.333227,0.934077,6.333227,0.934077,-,4.436521,0.862273,1260.808701,141.085905,1161.463704,...,3.67351,0.929993,5.839047,7.742985,-,-,NaN,NaN,NaN,NaN
VF74_136-3_101220_256s_20x30_a,4.066974,0.491437,4.066974,0.491437,-,1.986081,0.351852,598.329298,79.203812,584.73856,...,1.528126,0.361378,5.896663,5.254649,-,-,NaN,NaN,NaN,NaN
VF74_136-3_101220_256s_20x30_b,4.072526,0.490574,4.072526,0.490574,-,1.978682,0.350855,609.976438,81.700841,592.891491,...,1.418366,0.341097,7.237378,6.400376,-,-,NaN,NaN,NaN,NaN


In [8]:
fuegono = 0 

PATH = PATHS[fuegono]
FILES = glob.glob(PATH + "*")
FILES.sort()

MICOMP, THICKNESS = baselines.Load_ChemistryThickness(CHEMTHICK_PATH[fuegono])

DFS_FILES, DFS_DICT = baselines.Load_SampleCSV(FILES, wn_high = 5500, wn_low = 1000)
DF_OUTPUT, FAILURES = baselines.Run_All_Spectra(DFS_DICT, INPUT_PATHS[fuegono])
DF_OUTPUT.to_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[fuegono] + '_DF.csv')

# DF_OUTPUT = pd.read_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[fuegono] + '_DF.csv', index_col = 0)

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = baselines.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[fuegono] + '_H2OCO2.csv')
DENSITY_EPSILON.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[fuegono] + '_DensityEpsilon.csv')
MEGA_SPREADSHEET


::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  Multi-core Markov-chain Monte Carlo (mc3).
  Version 3.0.13.
  Copyright (c) 2015-2023 Patricio Cubillos and collaborators.
  mc3 is open-source software under the MIT license (see LICENSE).
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

Least-squares best-fitting parameters:
  [ 5.23364283e-01 -1.06184898e+00  5.08079324e-01 -1.87672707e-01
  2.01907538e-01  1.43095396e+03  2.35623242e+01  2.26234604e-02
  1.51695258e+03  4.00000000e+01  1.97532294e-02  2.47809786e-01
  4.59210479e-02 -1.28999055e-01 -3.66655504e-04  8.52450022e-01]

Yippee Ki Yay Monte Carlo!
Start MCMC chains  (Fri Mar  3 21:17:25 2023)

[:         ]  10.0% completed  (Fri Mar  3 21:17:31 2023)
Out-of-bound Trials:
[    3     1    10     0     0  2156 10329    31  3697 11847    62     0
     0     0     0     0]
Best Parameters: (chisq=41.4400)
[ 5.23364283e-01 -1.06184898e+00  5.08079324e-01 -1.87672707e-01
  2.019075

,H2OT_MEAN,H2OT_STD,H2OT_3550_M,H2OT_3550_STD,H2OT_3550_SAT,H2Om_1635_BP,H2Om_1635_STD,CO2_MEAN,CO2_STD,CO2_1515_BP,...,CO2_1430_BP,CO2_1430_STD,H2Om_5200_M,H2Om_5200_STD,OH_4500_M,OH_4500_STD,PH_5200_S2N,PH_4500_S2N,ERR_5200,ERR_4500
AC4_EUH102_030920_256s_15x20_a,2.711251,0.267147,2.711251,0.267147,-,1.03272,0.17238,253.95631,36.395826,236.755503,...,271.157117,53.038026,1.26629,0.279382,1.249255,0.299279,3.628526,4.414297,*,-
AC4_EUH102_030920_256s_15x20_b,2.716650,0.267846,2.71665,0.267846,-,1.036213,0.172948,282.566345,38.424868,279.489595,...,285.643095,54.793749,1.287352,0.282451,1.068926,0.256972,3.300287,4.74223,*,-
AC4_EUH102_030920_256s_15x20_c,2.740837,0.269627,2.740837,0.269627,-,1.034683,0.172663,256.557312,37.112831,262.505263,...,250.60936,53.509175,1.0545,0.273136,1.837307,0.436534,2.793253,4.524856,*,-
AC4_EUH33_030920_256s_20x20_a,1.386804,0.131167,1.386804,0.131167,-,0.237726,0.039672,96.503044,30.641365,55.266331,...,137.739757,51.970235,0.409572,0.082051,1.238835,0.290358,1.737557,5.13387,*,-
AC4_EUH33_030920_256s_20x20_b,1.391390,0.131604,1.39139,0.131604,-,0.239919,0.039998,89.680077,30.152411,47.064081,...,132.296072,51.581689,0.405357,0.081237,1.130578,0.249669,1.289942,5.637654,*,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC4_RH3_OL2_022120_256s_30x30_b,3.134082,0.289481,3.134082,0.289481,-,1.341127,0.228703,693.554621,64.859476,700.258482,...,686.850759,96.716961,1.160124,0.2732,1.298252,0.333057,4.11666,6.231784,-,-
AC4_RH3_OL2_022120_256s_30x30_c,3.168728,0.292897,3.168728,0.292897,-,1.34114,0.228845,700.668612,65.217086,705.357286,...,695.979938,97.897089,1.098824,0.257924,1.359424,0.3479,4.649161,5.146355,-,-
AC4_RH3_OL3_022120_256s_20x20_a,2.354622,0.189138,2.354622,0.189138,-,1.220121,0.179083,1180.853367,61.49121,1210.110408,...,1151.596326,86.686198,1.25872,0.238488,1.314938,0.275356,4.465166,8.194618,-,-
AC4_RH3_OL3_022120_256s_20x20_b,2.589241,0.168813,2.378397,0.191091,*,1.226871,0.179995,1165.835803,60.572702,1199.365136,...,1137.166219,85.519748,1.128131,0.218881,1.367766,0.285968,3.546042,6.591443,*,-


In [9]:
fuegorhno = 2 

PATH = PATHS[fuegorhno]
FILES = glob.glob(PATH + "*")
FILES.sort()

MICOMP, THICKNESS = baselines.Load_ChemistryThickness(CHEMTHICK_PATH[fuegorhno])

DFS_FILES, DFS_DICT = baselines.Load_SampleCSV(FILES, wn_high = 5500, wn_low = 1000)
DF_OUTPUT, FAILURES = baselines.Run_All_Spectra(DFS_DICT, INPUT_PATHS[fuegorhno])
DF_OUTPUT.to_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[fuegorhno] + '_DF.csv')

# DF_OUTPUT = pd.read_csv(path_beg + output_dir[-1] + '/' + OUTPUT_PATH[fuegorhno] + '_DF.csv', index_col = 0)

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = baselines.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[fuegorhno] + '_H2OCO2.csv')
DENSITY_EPSILON.to_csv(output_dir[-1] + '/' + OUTPUT_PATH[fuegorhno] + '_DensityEpsilon.csv')
MEGA_SPREADSHEET


::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  Multi-core Markov-chain Monte Carlo (mc3).
  Version 3.0.13.
  Copyright (c) 2015-2023 Patricio Cubillos and collaborators.
  mc3 is open-source software under the MIT license (see LICENSE).
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

Least-squares best-fitting parameters:
  [ 7.37787560e-01 -5.60018680e-01  4.62635951e-01  4.20236336e-02
  1.56984539e-01  1.42184213e+03  2.25000000e+01  1.04128798e-03
  1.53500000e+03  2.25000000e+01  5.24596018e-04  3.83359741e-01
 -1.33635200e-01  9.36261562e-03 -1.38172479e-04  7.44671901e-01]

Yippee Ki Yay Monte Carlo!
Start MCMC chains  (Fri Mar  3 22:58:10 2023)

[:         ]  10.0% completed  (Fri Mar  3 22:58:15 2023)
Out-of-bound Trials:
[    2     1     0     0     0 14123 13827  9553 14380  9661  9009     0
     0     0     0     0]
Best Parameters: (chisq=9.5069)
[ 7.37787560e-01 -5.60018680e-01  4.62635951e-01  4.20236336e-02
  1.5698453

,H2OT_MEAN,H2OT_STD,H2OT_3550_M,H2OT_3550_STD,H2OT_3550_SAT,H2Om_1635_BP,H2Om_1635_STD,CO2_MEAN,CO2_STD,CO2_1515_BP,...,CO2_1430_BP,CO2_1430_STD,H2Om_5200_M,H2Om_5200_STD,OH_4500_M,OH_4500_STD,PH_5200_S2N,PH_4500_S2N,ERR_5200,ERR_4500
FI45MI02a,3.183503,0.285251,3.542977,0.318535,*,1.439625,0.265592,6.044059,12.545542,4.033157,...,8.005546,18.800185,1.481934,0.418338,1.730864,0.504066,6.145607,4.434907,-,-
FI45MI02b,2.865085,0.246912,3.536198,0.317119,*,1.43242,0.264301,15.769772,13.229136,21.804003,...,9.494034,18.036265,1.698906,0.473149,1.410726,0.413242,8.076008,15.825501,-,-
FI45MI02c,3.108639,0.277173,3.48301,0.312446,*,1.439999,0.26584,13.909411,12.909779,17.960415,...,9.739806,17.818154,1.609148,0.448969,1.655387,0.483948,4.812446,7.790131,-,-
FI45MI08a,3.007218,0.272467,3.315223,0.276106,*,1.400681,0.258855,124.552058,11.151118,0.0,...,248.22016,21.826687,1.662027,0.48012,1.595866,0.477374,2.553545,5.505508,*,-
FI45MI08b,3.225859,0.300756,5.008988,0.416109,*,1.393668,0.257562,115.206631,10.084033,0.0,...,225.778746,19.323961,1.540286,0.455824,1.767307,0.528897,3.182398,7.39897,*,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FR06MI07b,4.141519,0.596018,4.141519,0.596018,-,1.824077,0.38497,2534.925791,251.63717,2607.17328,...,2462.678302,352.745388,1.21695,0.347996,3.438932,1.007607,0.931742,3.493894,*,*
FR06MI07c,4.033180,0.579372,4.03318,0.579372,-,1.855977,0.391183,2544.070351,253.176482,2683.155788,...,2404.984914,346.728988,2.754065,0.769262,3.079443,0.904258,1.627286,3.289291,*,*
FR06MI08a,3.691738,0.411185,3.691738,0.411185,-,1.696675,0.341729,2502.485829,174.097362,2516.409711,...,2488.561946,247.858323,1.183108,0.356988,0.762243,0.241739,3.235629,1.776139,*,*
FR06MI08b,3.637562,0.405778,3.637562,0.405778,-,1.679871,0.338002,2475.895418,172.332544,2491.731105,...,2460.059732,245.569631,1.401342,0.420066,0.98037,0.304602,3.504198,4.301674,*,-
